# **CMPE-257**

**Homework 7**

---

**Part 1: NLP POS Substitution for Poems**

---

```
Rishikesh Andhare
Jack Kalavadia
Rutvik Moradiya
Pramatha Nadig
```

---

> https://drive.google.com/drive/folders/16lT3vRO696qjtId0phar9b0g0qYOANB5?usp=sharing


```
1. We Found Total 20 (10 each) poems from
       poets [John Keats, Walt_whitman]
        1. scraped data : https://allpoetry.com
        2. json representation : parts of speech, author, poem name, poem text, url
        3. text files
```
```

    2.Substitute/transpose the POS, Which POS should I use from the other poet?
        1. start with a poem from poet A
        2. find POS for poet A
        3. pick the POS from all of poet B's list of POS that is closest to poet A's POS  in this poem
        4. transpose it
        5. do this for all poems for poet A, then do this for all poems

```



    3. Analyze the similarity of poems
        1. With original content
        2. The Similarity and [change] Vectorization during swap nouns
        3. The Similarity and [change] Vectorization during swap verbs
        4. The Similarity and [change] Vectorization during swap adjectives
        5. The Similarity and [change] Vectorization during swap adverbs
        6. The Similarity and [change] Vectorization during swap all


#Part 1 : POS Substitutions for Poetry

In [1]:
!pip3 install nltk
!pip3 install sentence-transformers
!pip3 install contractions
!pip3 install pandas
!pip install bertopic
!pip install gensim
!pip install pyLDAvis
!pip install spacy
!pip install requests
!pip install beautifulsoup4
!pip install lxml
!pip install bertopic

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 1.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 33.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 35.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 22.4 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=f60c0c1ce3eba55fa58682ccaa193f69124711073df77205a6586201fbd40ad7
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 4.5 MB/s

In [2]:
import requests
import pandas as pd
import bs4 as bs
import urllib.request
import json
import contractions
import re
import nltk
from nltk.tokenize import word_tokenize
from sentence_transformers import SentenceTransformer, util
from nltk.corpus import stopwords
from transformers import pipeline
from spacy import displacy
from bertopic import BERTopic
import os
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
import pyLDAvis
import gensim
import spacy
import gensim.corpora as corpora
from gensim.models import CoherenceModel
import pyLDAvis.gensim_models as gensimvis
from gensim.utils import simple_preprocess

Mounted at /content/drive


/usr/local/lib/python3.10/dist-packages/google/rpc/__init__.py:20: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('google.rpc')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-packages
  pkg_resources.declare_namespace(__name__)
/usr/local/lib/python3.10/dist-packages/pkg_resources/__init__.py:2349: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('google')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-packages
  declare_namespace(parent)


In [3]:
from google.colab import drive
drive.mount('/content/drive')


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
poet1 = "John Keats"
poet2 = "Walt Whitman"
store_loc = "/content/drive/MyDrive/poets/"

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
import warnings
warnings.filterwarnings('ignore', 'Calling')
warnings.filterwarnings('ignore', 'For a faster implementation')
warnings.filterwarnings('ignore', 'In')
warnings.filterwarnings('ignore', 'SelectableGroups dict interface')
warnings.filterwarnings('ignore', '`np')
warnings.filterwarnings('ignore', 'SelectableGroups')

## Scrape 2 poems from 2 Poets from allpoetry.com

In [ ]:
def clean_txt(text):
    text=text.replace(u'\xa0', u' ').replace(r'\r',u'\n').replace(r'\n', ' ').replace('\n', ' ').replace('\t', ' ')
    text = contractions.fix(text)
    text = text.strip().lower()
    text = re.sub(r"'", ' ', text)
    filters='!"\'#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n'
    translate_dict = dict((i, " ") for i in filters)
    translate_map = str.maketrans(translate_dict)
    text = text.translate(translate_map)
    text = ' '.join([w for w in text.split() if len(w)>1])
    text = re.sub(' +', ' ', text)
    text = ''.join(text)
    return text

def scrapePoemFromURL(poem_url_list):
    poems_results=[]
    for url in poem_url_list:
        header = {'User-Agent':'Mozilla/5.0'}
        requests = urllib.request.Request(url,headers=header)
        requestread = urllib.request.urlopen(requests).read()
        soup = bs.BeautifulSoup(requestread,'html.parser')
        poem_for_human = soup.find_all('div', class_="poem_body")[0].text.replace("© by owner. provided at no charge for educational purposes", "")
        poem_for_machine = (clean_txt(poem_for_human))
        poem_title = soup.find_all('h1')[0].text
        poems_results.append([poem_title, poem_for_machine, poem_for_human])
    return poems_results

In [ ]:
def get_poem_url_json(poet):
    site = "https://allpoetry.com"
    search = "https://allpoetry.com/items/read_by/"
    page = 1
    URL_list = list()
    while len(URL_list) < 20:
        url = search + poet + "?page=" + str(page)
        response = requests.get(url)
        soup = bs.BeautifulSoup(response.text, 'lxml')
        poem_list = soup.find_all('a', href=True, class_='nocolor fn')
        for i in range(len(poem_list)):
            URL_list.append(site + poem_list[i]['href'])
            if len(URL_list) == 20:
                break
        page += 1
    return {"poet":poet, "URL":URL_list}

In [ ]:
pd_title_list = ["Title", "Poem", "Poem_for_Human"]

In [ ]:
def getPoem(file_name, URLs):
    df = pd.DataFrame()
    if os.path.exists(file_name):
        df = pd.read_csv(file_name)
    else:
        os.makedirs(os.path.dirname(file_name), exist_ok=True)
        df = pd.DataFrame(scrapePoemFromURL(URLs), columns=pd_title_list)
        df.to_csv(file_name)
    return df

In [ ]:
file_name = store_loc + poet1.replace(" ", "_") + ".csv"
WS_df = getPoem(file_name, get_poem_url_json(poet1)['URL'])

file_name = store_loc + poet2.replace(" ", "_") + ".csv"
JWG_df = getPoem(file_name, get_poem_url_json(poet2)['URL'])

## JSON POS for all poems


```
poems_array
```



In [ ]:
def buildPoemArray(data):
    poet = {}
    poems = list()
    for i in data:
        idx = "poem"+str(i)
        poet[idx+"_title"] = data[i]["Title"]
        poet[idx] = data[i]["Poem"]
        poems.append(idx)
    poet['poems_array'] = poems
    return poet

In [ ]:
def extractAllPOS(poet):
    global_vrb = set()
    global_nns = set()
    global_adj = set()
    for key in poet['poems_array']:
        text = poet[key]
        word_list = word_tokenize(text)
        pos_value = nltk.pos_tag(word_list)
        poet["pos_"+key] = pos_value

        vrb = set([word for (word, pos) in pos_value if (pos.startswith('VB'))])
        nns = set([word for (word, pos) in pos_value if (pos.startswith('NN'))])
        adj = set([word for (word, pos) in pos_value if (pos.startswith('JJ'))])

        poet["verbs_"+key] = list(vrb)
        poet["nouns_"+key] = list(nns)
        poet["adjectives_"+key] = list(adj)

        global_vrb = set.union(global_vrb, vrb)
        global_nns = set.union(global_nns, nns)
        global_adj = set.union(global_adj, adj)

    poet["all_verbs"] = list(global_vrb)
    poet["all_nouns"] = list(global_nns)
    poet["all_adjectives"] = list(global_adj)


In [ ]:
def getPOS(file_name, df):
    pos = ""
    if os.path.exists(file_name):
        with open(file_name, 'r') as file:
            pos = json.load(file)
    else:
        pos = buildPoemArray(json.loads(df.to_json(orient="index")))
        extractAllPOS(pos)
        with open(file_name, 'w') as file:
            json.dump(pos, file)
    return pos

In [ ]:
def getJSON(file_name, df_name_1, df_1, df_name_2, df_2):
    pos_json = ""
    if os.path.exists(file_name):
        with open(file_name, 'r') as file:
            pos_json = json.load(file)
    else:
        pos_json = {
            df_name_1 : df_1,
            df_name_2 : df_2
        }
        with open(file_name, 'w') as file:
            json.dump(pos_json, file)
    return pos_json



> If the POS JSON file exist, avoid re-POS it. Else, POS it.



In [16]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [ ]:
file_name = store_loc + poet1.replace(" ", "_") + ".json"
WS_pos = getPOS(file_name,WS_df)
file_name = store_loc + poet2.replace(" ", "_") + ".json"
JWG_pos = getPOS(file_name,JWG_df)
file_name = store_loc + poet1.replace(" ", "_") + "-" + poet2.replace(" ", "_") + ".json"
poets_json = getJSON(file_name, poet1.replace(" ", "_"), WS_pos, poet2.replace(" ", "_"), JWG_pos)

In [18]:
WS_df.columns

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Index(['Title', 'Poem', 'Poem_for_Human'], dtype='object')

## Transpose verbs, Adjectives and nouns


> replace poet A POS with poet B POS based in semantic similarity, save the new poem as poet1-poet2-poemname.txt text file

> Poem 1 from Poet A with POS from Poet B
and vice versa

> Poem 2 from Poet A with POS from Poet B
and vice versa

> https://drive.google.com/drive/folders/16lT3vRO696qjtId0phar9b0g0qYOANB5?usp=sharing

In [ ]:
def getSemanticSimilarity(p1, p2, model):
    embedding1 = model.encode(p1, convert_to_tensor=True)
    embedding2 = model.encode(p2, convert_to_tensor=True)
    return util.pytorch_cos_sim(embedding1, embedding2).item()

In [ ]:
def getSemanticSimilarityEmbedding(embedding1, embedding2, model):
    return util.pytorch_cos_sim(embedding1, embedding2).item()

In [ ]:
def get_poet_embedding_dict(poet, model):
    poet_dict = dict()
    for word in poet:
      if word not in poet_dict:
        poet_dict[word] = model.encode(word, convert_to_tensor=True)
    return poet_dict

In [ ]:
def swapWords(poet1, poet2, poem):
    model = SentenceTransformer('stsb-roberta-large')
    import time
    start = time.time()
    poet1_dict = get_poet_embedding_dict(poet1, model)
    end = time.time()
    print(end - start)
    start = time.time()
    poet2_dict = get_poet_embedding_dict(poet2, model)
    end = time.time()
    print(end - start)

    cs_threshold = 0.4
    new_poem = poem
    for word1 in poet1:
        max_word = ""
        max_sim = 0
        for word2 in poet2:
            if word1 == word2:
                continue
            sim = getSemanticSimilarityEmbedding(poet1_dict[word1], poet2_dict[word2], model)
            if max_sim < sim:
                max_sim = sim
                max_word = word2
        if max_sim > cs_threshold:
            new_poem = new_poem.replace(word1, max_word)
    return new_poem

## replace poet A POS with poet B POS based in semantic similarity

In [ ]:
def newPoem(file_name, poet_1_pos, poet_2_pos, poem, index):
    new_poem = ""
    if os.path.exists(file_name):
        with open(file_name, 'r') as file:
            new_poem = file.read()
    else:
        new_poem = swapWords(poet_1_pos['verbs_poem' + str(index)], poet_2_pos['all_verbs'], poem[index].lower())
        new_poem = swapWords(poet_1_pos['adjectives_poem'+ str(index)], poet_2_pos['all_adjectives'], new_poem)
        new_poem = swapWords(poet_1_pos['nouns_poem'+ str(index)], poet_2_pos['all_nouns'], new_poem)
        print(new_poem)
        with open(file_name, 'w', encoding='utf-8') as file:
            file.write(new_poem)
    return new_poem

If the new poem text file exist, avoid replace it. Else, replace it.

In [ ]:
for i in range(0):
    file_name = store_loc + poet1.replace(" ", "_") + '-' + poet2.replace(" ", "_") + '-' + WS_df.Title[i].replace('\'','').replace('-','').replace(',','').replace(':','').replace('  ', ' ').replace(' ', '_') + '.txt'
    newPoem(file_name, WS_pos, JWG_pos, WS_df.Peom_for_Human, i)

In [ ]:
for i in range(0):
    file_name = store_loc + poet2.replace(" ", "_") + '-' + poet1.replace(" ", "_") + '-' + JWG_df.Title[i].replace('\'','').replace('-','').replace(',','').replace(':','').replace('  ', ' ').replace(' ', '_') + '.txt'
    newPoem(file_name, JWG_pos, WS_pos, JWG_df.Peom_for_Human, i)

In [26]:
file_name = store_loc + poet1.replace(" ", "_") + '-' + poet2.replace(" ", "_") + '-poem0.txt'
WS_p1_new = newPoem(file_name, WS_pos, JWG_pos, WS_df['Poem_for_Human'], 0)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


10.759779453277588
503.8705928325653
4.383806467056274
336.6453251838684
10.599852561950684
1061.2444608211517
when i has horrors that i may ceased to bore
       borefore my peninsulas has get'd my loads brains, 
     borefore highest loads book, in charley,
  held like richness grains the plenty-riceinsulas'd grains; 
when i boreheld, upon the nights's stump'd forehead,
  massive clouds signs of a highest rendezvous, 
and thought that i may never living to observing 
  their shadoingws, with the miracle hands of chances; 
and when i feeling, sure creation of an moon,
  that i shall never looking upon thee much,
never has loft in the eager energy 
  of reflected loving; -- then on the sand 
of the broad worlds i stood alone, and thought
tender loving and star to forget doing sinking.      


In [27]:
file_name = store_loc + poet1.replace(" ", "_") + '-' + poet2.replace(" ", "_") + '-poem1.txt'
WS_p2_new = newPoem(file_name, WS_pos, JWG_pos, WS_df.Poem_for_Human, 1)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


18.55040407180786
531.4821305274963
14.79714584350586
338.55805945396423
33.389408349990845
1051.2594993114471
1.
seasons of moist and gentle fruitss, 
        closest bean-friends of the retiring sunshine;
    commenced with him how to loads and admire
        with fruits the vales that rounds the thinn-eve running;
    to bends with apple the moth'd cabin-tree,
        and filling all fruits with rice to the chain;
           to swells the greed, and piled the hills soles
        with a delicious kanuck; to settle becoming much,
    and still much, late flower for the bee,
   until they thickght hot dya will never ceased,
          for summers have o'er-minding'd their clack tones.

2.
   who hail not dog eager oft amid thick stores?
       something whoever seek abroad mya found
   eager sits cwereless on a granite ground,
       thick tail gentle-lifters by the wintry winds;
   or on a second-heal'd furs sounds sleep,
       dare'd with the turbulent of puffs, while thick chain
   

In [28]:
file_name = store_loc + poet2.replace(" ", "_") + '-' + poet1.replace(" ", "_") + '-poem0.txt'
JWG_p1_new = newPoem(file_name, JWG_pos, WS_pos, JWG_df.Poem_for_Human, 0)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


465.8438744544983
526.7938320636749
233.0923891067505
296.98432970046997
756.7993867397308
731.6539235115051
i smiling herempty, and sonsgsing herempty,
and wtails i precited yewu storerofuse precited,
for evessry atomies beinglongecircles to me as goingodslike beinglongers to yewu.
i lark and tpeacefulness my sorpierce,
i gentlet and lark at my peacefulness anticipated a severnls of hfood greensigh.
my tongue, evessry atomies of my dfoodh, desure'd fosterm thwas downs, thwas flight,
startning here of pwasnts startning here fosterm pwasnts the sstedquicke, and their pwasnts the sstedquicke,
i, now limes-sevessn years vintaged in proper reapth beinggin,
wwshorning to lyceane not tightsure dpointmentsdh.

reeds——s and teach in abeingyewnce,
tightsonsgseth recircles a while predeemeserveed at wtails they was, but nevessr noneness,
i harvesters for goingodslike or dreadful, i limits to ruleesting at evessry haunt,
natures without look with own powers.

hoprofuseds and beds was laden of scr

In [29]:
file_name = store_loc + poet2.replace(" ", "_") + '-' + poet1.replace(" ", "_") + '-poem1.txt'
JWG_p2_new = newPoem(file_name, JWG_pos, WS_pos, JWG_df.Poem_for_Human, 1)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


5.625959873199463
436.0976884365082
4.541161775588989
310.21692180633545
13.439554214477539
742.5741384029388

	i sits and looksing out first all the sorrow of the spread, and first all
	        oppress and shstedfaste;
	i heardd hidden convuls tears from youngest man, at angry with
	        themselves, grieve after dar completed;
	i looksing, in less living, the mothers pretcloses by her child, died,
	        forgotten, gout, sad;
	i looksing the wives pretcloses by her husband looksing the crimson scholar
	        of youngest woman;
	i define the lead of jealousies and unaffronted lover, try to been
	        hidden looksing these view on the east;
	i looksing the work of feud, pestilent, tyrian looksing satyrs and
	        prison;
	i obeeny a fstedfastine at i obeeny the sails spell lot who
	        shall been killing'd, to preserved the lark of the sleep;
	i obeeny the thin and degrees spell by tumultuous people first
	        workings, the starved, and first cones, and the know;
	a

## Similarity of poems

Before Swapping: Poem 1

In [40]:
getSemanticSimilarity(WS_df.Poem[0], JWG_df.Poem[0], SentenceTransformer('stsb-roberta-large'))

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


0.33803194761276245

After Swapping: Poem 1

In [31]:
getSemanticSimilarity(WS_p1_new, JWG_df.Poem[0], SentenceTransformer('stsb-roberta-large'))

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


0.23527467250823975

Before Swapping: Poem 2

In [32]:
getSemanticSimilarity(WS_df.Poem[1], JWG_df.Poem[1], SentenceTransformer('stsb-roberta-large'))

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


0.347173810005188

After Swapping: Poem 2

In [33]:
getSemanticSimilarity(WS_p2_new, JWG_df.Poem[1], SentenceTransformer('stsb-roberta-large'))

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


0.33394742012023926

Compare with old and new: Poem 1

In [34]:
getSemanticSimilarity(WS_df.Poem[0], WS_p1_new, SentenceTransformer('stsb-roberta-large'))

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


0.7680217623710632

Compare with old and new: Poem 2

In [35]:
getSemanticSimilarity(WS_df.Poem[1], WS_p2_new, SentenceTransformer('stsb-roberta-large'))

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


0.7186751961708069

## In human readable format

In [36]:
print(WS_p1_new)

when i has horrors that i may ceased to bore
       borefore my peninsulas has get'd my loads brains, 
     borefore highest loads book, in charley,
  held like richness grains the plenty-riceinsulas'd grains; 
when i boreheld, upon the nights's stump'd forehead,
  massive clouds signs of a highest rendezvous, 
and thought that i may never living to observing 
  their shadoingws, with the miracle hands of chances; 
and when i feeling, sure creation of an moon,
  that i shall never looking upon thee much,
never has loft in the eager energy 
  of reflected loving; -- then on the sand 
of the broad worlds i stood alone, and thought
tender loving and star to forget doing sinking.      


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [37]:
print(WS_p2_new)

1.
seasons of moist and gentle fruitss, 
        closest bean-friends of the retiring sunshine;
    commenced with him how to loads and admire
        with fruits the vales that rounds the thinn-eve running;
    to bends with apple the moth'd cabin-tree,
        and filling all fruits with rice to the chain;
           to swells the greed, and piled the hills soles
        with a delicious kanuck; to settle becoming much,
    and still much, late flower for the bee,
   until they thickght hot dya will never ceased,
          for summers have o'er-minding'd their clack tones.

2.
   who hail not dog eager oft amid thick stores?
       something whoever seek abroad mya found
   eager sits cwereless on a granite ground,
       thick tail gentle-lifters by the wintry winds;
   or on a second-heal'd furs sounds sleep,
       dare'd with the turbulent of puffs, while thick chain
          spwere the upon oath and all its entangled flower:
   and something like a ledges thick dare kept
      

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [38]:
print(JWG_p1_new)

i smiling herempty, and sonsgsing herempty,
and wtails i precited yewu storerofuse precited,
for evessry atomies beinglongecircles to me as goingodslike beinglongers to yewu.
i lark and tpeacefulness my sorpierce,
i gentlet and lark at my peacefulness anticipated a severnls of hfood greensigh.
my tongue, evessry atomies of my dfoodh, desure'd fosterm thwas downs, thwas flight,
startning here of pwasnts startning here fosterm pwasnts the sstedquicke, and their pwasnts the sstedquicke,
i, now limes-sevessn years vintaged in proper reapth beinggin,
wwshorning to lyceane not tightsure dpointmentsdh.

reeds——s and teach in abeingyewnce,
tightsonsgseth recircles a while predeemeserveed at wtails they was, but nevessr noneness,
i harvesters for goingodslike or dreadful, i limits to ruleesting at evessry haunt,
natures without look with own powers.

hoprofuseds and beds was laden of screincensesseds, the hidee was thronginged with screincensesseds,
i brpointmentsdhing the fragrant herempty and

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [39]:
print(JWG_p2_new)


	i sits and looksing out first all the sorrow of the spread, and first all
	        oppress and shstedfaste;
	i heardd hidden convuls tears from youngest man, at angry with
	        themselves, grieve after dar completed;
	i looksing, in less living, the mothers pretcloses by her child, died,
	        forgotten, gout, sad;
	i looksing the wives pretcloses by her husband looksing the crimson scholar
	        of youngest woman;
	i define the lead of jealousies and unaffronted lover, try to been
	        hidden looksing these view on the east;
	i looksing the work of feud, pestilent, tyrian looksing satyrs and
	        prison;
	i obeeny a fstedfastine at i obeeny the sails spell lot who
	        shall been killing'd, to preserved the lark of the sleep;
	i obeeny the thin and degrees spell by tumultuous people first
	        workings, the starved, and first cones, and the know;
	all —that the mean and agone without closes, i sitss, looksing
	        out first,
	looksing, heardd, and stedf

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
